In [1]:
import pandas as pd
import numpy as np


In [2]:
from metrics_f1 import calc_f1_score

## Загрузка данных

In [3]:
path_train = r"./"

In [4]:
# данные по дислокации
dislok = pd.read_parquet(path_train + '/dislok_wagons.parquet').convert_dtypes()
# данные по текущим ремонтам
pr_rem = pd.read_parquet(path_train + '/pr_rems.parquet').convert_dtypes()
# список вагонов с остаточным пробегом на момент прогноза
wag_prob = pd.read_parquet(path_train + '/wagons_probeg_ownersip.parquet').convert_dtypes()
 # параметры вагона
wag_param = pd.read_parquet(path_train + '/wag_params.parquet').convert_dtypes()
# таргет по прогноза выбытия вагонов в ПР на месяц и на 10 дней
target = pd.read_csv(path_train +'/target/y_train.csv').convert_dtypes()
 # текущие ремонты вагонов
tr_rem = pd.read_parquet(path_train + '/tr_rems.parquet').convert_dtypes()

In [5]:
wag_param = wag_param.drop_duplicates(subset='wagnum', keep='last')# у вагонов могут меняться параметры, поэтмоу номер дублируется. В данной модели это фактор не учитывается

In [6]:
month_to_predict = pd.to_datetime('2022-12-01')

In [7]:
target.month = pd.to_datetime(target.month)
target = target[target.month == month_to_predict][['wagnum','target_month','target_day']]

In [8]:
target.target_month.sum(), target.target_day.sum()

(1584, 570)

# Наивная модель

Наивная модель будет построена на правилах с использованием минимального набора данных, без применения Ml.

Реальный процесс выглядит следующим образом - в начале месяца берется срез по парку по всем вагонам, за ремонт которых несёт ответственность ПГК. Для выбранных вагонов требуется установить, какие из них будут отремонтированы в текущем месяце. Данная информация помогает планировать нагрузку на вагоно-ремонтное предприятие(ВРП). Вторая модель определяет критичные вагоны, которые будут отправлены в ремонт в первую очередь( в ближайшие 10 дней). Это помогает фокусировать внимание диспетчеров.

Основными критериями по которым вагон отправляется в плановый ремонт - является его остаточный пробег и срок до планового ремонта.
В регламентах РЖД используется следующее правило - если ресурс по пробегу не превышает 500 км и/или плановый ремонт должен наступить через 15 дней(или меньше), то вагон может ехать только на ВРП.
Из этого регламента вытекают две особенности:
1. Диспетчер старается отправить вагон раньше положенных значений. Это позволяет выбрать предприятия, на которых ремонтироваться дешевле, а не ближайшее.
2. Компания-оператор может выбирать какому из нормативов нужно следовать - ремонтировать вагон по сроку, или по пробегу, или по обоим критериям сразу. Поэтому встречаются вагоны, у которых пробег может не отслеживаться.

Вагон может быть отправлен в плановый ремонт и раньше положенного. На это может влиять, например, история грузовых операций и количество текущих(мелких) ремонтов. Основная цель участников в данной задаче - найти закономерности и оценить значимые признаки, по которым вагон выбывает в плановый ремонт.

## Подготовка данных

In [9]:
# оставим только данные по остаточному пробегу для каждого номерав вагона
wag_prob = wag_prob[(wag_prob.repdate == month_to_predict) | (wag_prob.repdate == wag_prob.repdate.min())]

In [10]:
# оценим среднесуточный пробег из данных по пробегу вагона, на тот случай, если данных по нормативу нет
wag_prob_ =wag_prob.groupby('wagnum', as_index = False).agg({'repdate':['max', 'min'] , 'ost_prob': ['max','min']},)#.droplevel(1)
wag_prob_.columns = [head+'_' + name
                     if head!='wagnum'
                     else head
                     for head, name in wag_prob_.columns ]

wag_prob_['diff_days'] = wag_prob_.repdate_max - wag_prob_.repdate_min
wag_prob_['mean_run'] = (wag_prob_.ost_prob_max - wag_prob_.ost_prob_min )/ wag_prob_.diff_days.dt.days
wag_prob = wag_prob[wag_prob.repdate == wag_prob.repdate.max()][['wagnum','ost_prob']]
wag_prob = wag_prob.merge(wag_prob_[['wagnum','mean_run']])

In [11]:
# для каждого вагона оставим только информацию по сроку службы и нормативу суточного пробега между ПР
wag_param = wag_param[['wagnum','srok_sl','cnsi_probeg_dr','cnsi_probeg_kr']]

In [12]:
# добавим признак, что вагон был в ПР в предыдущем месяце. Скорее всего, если вагон был в ПР недавно, то повторно он не поедет
pr_rem['was_repair_in_prev_month'] = 1
pr_rem = pr_rem[['wagnum','was_repair_in_prev_month']]
pr_rem = pr_rem.drop_duplicates(subset='wagnum') #некоторые вагоны все же ремонтируются больше 1 раза, поэтому нужен сбросить дубли

In [13]:
# посчитаем сколько текущих ремонтов было за прошедший период
tr_rem = tr_rem.groupby('wagnum', as_index= False).kod_vrab.count()

In [14]:
# сохраним только дату следующего планового ремонта для вагона
dislok = dislok[['wagnum','date_pl_rem']].drop_duplicates(subset = 'wagnum', keep='last')

In [15]:
# соберем все данные вместе
wp = target[['wagnum']].merge(wag_param, on ='wagnum', how = 'left')\
             .merge(wag_prob, how = 'left')\
             .merge(pr_rem, how = 'left')\
             .merge(tr_rem, how = 'left')\
             .merge(dislok, how = 'left')

In [16]:
wp.head()

,wagnum,srok_sl,cnsi_probeg_dr,cnsi_probeg_kr,ost_prob,mean_run,was_repair_in_prev_month,kod_vrab,date_pl_rem
0,33361,2033-03-01,110,160,159916,1248.97541,1.0,3.0,2023-02-17
1,33364,2031-04-12,110,160,4268,269.139344,1.0,2.0,2023-10-03
2,33366,2032-01-21,110,160,1507,71.598361,1.0,2.0,2023-04-03
3,33358,2032-11-30,110,160,30223,95.114754,NaN,2.0,2024-02-23
4,33349,2033-12-04,110,160,153839,1214.07377,1.0,NaN,2023-07-06
...,...,...,...,...,...,...,...,...,...
33972,25045,2034-08-30,110,160,150682,1086.07377,1.0,NaN,2023-06-29
33973,27156,2030-12-18,110,160,153800,1170.663934,1.0,4.0,2023-06-27
33974,21361,2028-03-04,160,160,141836,1038.860656,1.0,NaN,2023-06-08
33975,8061,2025-10-10,110,160,156895,NaN,1.0,NaN,2022-10-08


In [17]:
# Получим среднесуточный пробег, как среднее от нормативов и реального пробега
wp[['cnsi_probeg_dr','cnsi_probeg_kr','mean_run']] = wp[['cnsi_probeg_dr','cnsi_probeg_kr','mean_run']].fillna(0)
wp['day_run'] = wp.apply(lambda x : [ val  for val in [x.cnsi_probeg_kr, x.cnsi_probeg_dr, x.mean_run] if val != 0], axis = 1 )
wp['day_run']= wp.apply(lambda x : np.mean(x.day_run) if len(x.day_run)> 0 else 0, axis = 1 )

In [18]:
wp['current_date'] = month_to_predict

In [19]:
# определим, сколько дней осталось до истечения срока службы
wp['date_diff_srk_sl'] = wp['srok_sl']- wp['current_date']

In [20]:
# определим, сколько дней осталось до ближайшего ПР
wp['date_diff_pl_rem'] = wp['date_pl_rem']- wp['current_date']

In [21]:
# определим, какой остаточный ресурс будет на момент окончания месяца
wp['prob_end_month'] = wp['ost_prob'] - wp['day_run']* 30

In [22]:
wp['target_month'] = 0

In [23]:
# вагон выбывает в ПР в следующем месяце, если:
# остаточный пробег < 5 000 км
# срок службы < 500 лней
# до следующего  ПР < 40 дней
# ,число текущих ремонтов > 5
wp.loc[(wp.prob_end_month <= 5000) \
       | (wp.date_diff_srk_sl < pd.to_timedelta(40))\
        | (wp.date_diff_pl_rem < pd.to_timedelta(10)) \
        | (wp.kod_vrab > 5),'target_month'] = 1

In [24]:
wp['target_day'] = wp['target_month']

In [25]:
pred_target = target[['wagnum']].merge(wp[['wagnum','target_month','target_day']],how = 'left')
pred_target = pred_target.drop_duplicates(subset = 'wagnum')

In [26]:
# Проверим соотношение отмеченных вагонов с фактическим значением
round(pred_target.target_month.sum() / target.target_month.sum(), 2)

3.5

In [27]:
# сохраним таргет за месяц  для выбранного периода отдельно
target_path = './prediction/target_predicton.csv'

In [28]:
pred_target.drop_duplicates(subset = 'wagnum').to_csv(target_path, index=False)

In [29]:
true_target_path = './prediction/target_predicton_true.csv'

In [30]:
target.drop_duplicates(subset = 'wagnum').to_csv(true_target_path, index=False)

In [31]:
# оценим насколько хорошо удалось предсказать выбытие вагонов  по месяцу и по 10 дням
calc_f1_score( true_target_path, target_path,)

0.23121339736861674